The network predicts quality scores for the individual repetitions for Exercise 1 - Deep Squat. The input to the network are the raw measurement data with 117 dimensions.

In [1]:
# Import libraries and functions

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

import csv
import os,random

# The code is run on a CPU

from keras.models import Model
from keras.layers import Input, Conv1D, LSTM, Dense, Dropout, Activation, Flatten, concatenate, UpSampling1D
from keras.callbacks import EarlyStopping
from keras.optimizers import *
from keras.layers import Lambda

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
from math import sqrt

import datetime
now = datetime.datetime.now

In [2]:
timesteps = 240  # number of timesteps
nr = 90   # number of repetitions
n_dim = 117  # dimension of the data sequences

In [3]:
import DataViconLoad   # Import the data
Correct_data, Correct_label, Incorrect_data, Incorrect_label = DataViconLoad.load_data()

# Print the size of the data 
print(Correct_data.shape, 'correct sequences')
print(Correct_label.shape, 'correct labels')
print(Incorrect_data.shape, 'incorrect sequences')
print(Incorrect_label.shape, 'incorrect labels')

(90, 240, 117) correct sequences
(90, 1) correct labels
(90, 240, 117) incorrect sequences
(90, 1) incorrect labels


In [4]:
len(Correct_data[0])

240

In [5]:
# Split the data into training and validation sets
# Training set: 70%
# Validation set: 30%

# Sample random indices
trainidx1 = random.sample(range(0,Correct_data.shape[0]),int(nr*0.7))
trainidx2 = random.sample(range(0,Incorrect_data.shape[0]),int(nr*0.7))
valididx1 = np.setdiff1d(np.arange(0,nr,1),trainidx1)
valididx2 = np.setdiff1d(np.arange(0,nr,1),trainidx2)

# Training set: data and labels
train_x = np.concatenate((Correct_data[trainidx1,:,:],Incorrect_data[trainidx2,:,:]))
print(train_x.shape, 'training data')
train_y = np.concatenate((np.squeeze(Correct_label[trainidx1]),np.squeeze(Incorrect_label[trainidx2])))
print(train_y.shape, 'training labels')

# Validation set: data and labels
valid_x = np.concatenate((Correct_data[valididx1,:,:],Incorrect_data[valididx2,:,:]))
print(valid_x.shape, 'validation data')
valid_y = np.concatenate((np.squeeze(Correct_label[valididx1]),np.squeeze(Incorrect_label[valididx2])))
print(valid_y.shape, 'validation labels')

(124, 240, 117) training data
(124,) training labels
(56, 240, 117) validation data
(56,) validation labels


In [6]:
# Plot the first and last sequence in the training and validation sets
plt.figure(figsize = (14,4))
plt.subplot(2,2,1)
plt.plot(train_x[0])
plt.title('First Train Sequence')
plt.subplot(2,2,2)
plt.plot(train_x[-1])
plt.title('Last Train Sequence')
plt.subplot(2,2,3)
plt.plot(valid_x[0])
plt.title('First Validation Sequence')
plt.subplot(2,2,4)
plt.plot(valid_x[-1])
plt.title('Last Validation Sequence')
plt.tight_layout()
plt.show()

In [11]:
from tcn import TCN
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

#tcn_layer = TCN(input_shape=(timesteps, n_dim))
tcn_layer = TCN(input_shape=(timesteps, n_dim), nb_filters=64,kernel_size=3,nb_stacks=1,dilations=(1, 2, 4, 8, 16, 32),
    padding='causal',use_skip_connections=True,dropout_rate=0.01,return_sequences=False,
    activation='relu',kernel_initializer='he_normal',use_batch_norm=False,use_layer_norm=False,use_weight_norm=False)

tcn_layer_1 = TCN(input_shape=(timesteps, 65), nb_filters=32,kernel_size=3,nb_stacks=1,dilations=(1, 2, 4, 8, 16, 32),
    padding='causal',use_skip_connections=True,dropout_rate=0.01,return_sequences=False,
    activation='relu',kernel_initializer='he_normal',use_batch_norm=False,use_layer_norm=False,use_weight_norm=False)

# The receptive field tells you how far the model can see in terms of timesteps.
print('Receptive field size =', tcn_layer.receptive_field)

Receptive field size = 253


In [12]:
m = Sequential([
    tcn_layer,
    tcn_layer_1,
    Dense(1)
])

m.summary()

ValueError: Exception encountered when calling layer "tcn_4" (type TCN).

in user code:

    File "E:\Programs\ProgramFiles\anaconda3\envs\edsPyR\lib\site-packages\tcn\tcn.py", line 344, in call  *
        x, skip_out = res_block(x, training=training)
    File "E:\Programs\ProgramFiles\anaconda3\envs\edsPyR\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "D:\Users\sjw19206\AppData\Local\Temp\__autograph_generated_file5g0piw9j.py", line 30, in tf__call
        ag__.for_stmt(ag__.ld(self).layers, None, loop_body, get_state, set_state, ('x1',), {'iterate_names': 'layer'})
    File "D:\Users\sjw19206\AppData\Local\Temp\__autograph_generated_file5g0piw9j.py", line 27, in loop_body
        x1 = ag__.if_exp(ag__.ld(training_flag), lambda : ag__.converted_call(ag__.ld(layer), (ag__.ld(x1),), dict(training=ag__.ld(training)), fscope), lambda : ag__.converted_call(ag__.ld(layer), (ag__.ld(x1),), None,

In [ ]:
m.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')

#tcn_full_summary(m, expand_residual_blocks=False)


#m.fit(train_x, train_y, epochs=10, validation_split=0.2)

t = now()
    
early_stopping = EarlyStopping(monitor='val_loss', patience = 25)

history = m.fit(train_x, train_y, batch_size=10, epochs=500, verbose=0, 
                validation_data=(valid_x, valid_y), callbacks = [early_stopping])

print('Training time: %s' % (now() - t))

In [ ]:
# Plot the results
plt.figure(1)
plt.plot(history.history['loss'], 'b', label = 'Training Loss')
plt.title('Training Loss')
plt.plot(history.history['val_loss'], 'r', label = 'Validation Loss')
plt.legend()
plt.tight_layout()
plt.show()
# Print the minimum loss
print("Training loss", np.min(history.history['loss']))
print("Validation loss",np.min(history.history['val_loss']))    

In [ ]:
# Plot the prediction of the model for the training and validation sets
pred_train = m.predict(train_x)

pred_test = m.predict(valid_x)

plt.figure(figsize = (8,8))
plt.subplot(2,1,1)
plt.plot(pred_train,'s', color='red', label='Prediction', linestyle='None', alpha = 0.5, markersize=6)
plt.plot(train_y,'o', color='green',label='Quality Score', alpha = 0.4, markersize=6)
plt.ylim([-0.1,1.1])
plt.title('Training Set',fontsize=18)
plt.xlabel('Sequence Number',fontsize=16)
plt.ylabel('Quality Scale',fontsize=16)
plt.legend(loc=3, prop={'size':14}) # loc:position
plt.subplot(2,1,2)
plt.plot(pred_test,'s', color='red', label='Prediction', linestyle='None', alpha = 0.5, markersize=6)
plt.plot(valid_y,'o', color='green',label='Quality Score', alpha = 0.4, markersize=6)
plt.title('Testing Set',fontsize=18)
plt.ylim([-0.1,1.1])
plt.xlabel('Sequence Number',fontsize=16)
plt.ylabel('Quality Scale',fontsize=16)
plt.legend(loc=3, prop={'size':14}) # loc:position
plt.tight_layout()
plt.savefig('../../Results/SpatioTemporalNN_Vicon_Scores.png', dpi=300)
plt.show()

In [ ]:
# Calculate the cumulative deviation and rms deviation for the validation set
test_dev = abs(np.squeeze(pred_test)-valid_y)
# Cumulative deviation
mean_abs_dev = np.mean(test_dev)
# RMS deviation
rms_dev = sqrt(mean_squared_error(pred_test, valid_y))
print('Mean absolute deviation:', mean_abs_dev)
print('RMS deviation:', rms_dev)
print('test_dev:', test_dev)